# Load single graph data
이는 단일 구조물의 graph data로, dictionary와 유사한 구조로 접근한다.

- x: node features
- edge_index: node pairs
- edge_attr: edge features
- Acceleration, Velocity, Desplacement, Moment_Z, Shear_Y, 원시 반응 시간이력 데이터의 크기는 모두 [2000, 8]，반응 시간이력의 샘플링 간격 0.05，최대 길이를 100초로 설정했으므로 2000 스텝이 됨. 8은 예측 가능한 최대 건물 층수.
- ground_motion (mm/$s^2$): [time_steps]
- time_steps: 전처리되지 않은 ground motion의 스텝 수
- sample_rate: 지표 가속도 데이터의 취득 주기(반응 시간이력 샘플링 주기의 10배)
- ground_motion_name: 지진 기록명

In [7]:
import torch
from torch import nn
sample_path = "./Data/Linear_Analysis/eval/ChiChi/structure_401/structure_graph.pt"
sample_graph = torch.load(sample_path, weights_only=False)
print(sample_graph)

Data(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], Acceleration=[2000, 8], Velocity=[2000, 8], Displacement=[2000, 8], Moment_Z=[147, 2000, 6], Shear_Y=[147, 2000, 6], ground_motion=[6000], time_steps=6000, sample_rate=0.005, ground_motion_name='1999.09.21_01.47.159_Chichi_TCU011_A900_90.0')


# Load Data with GraphDataset
로드할 데이터셋 경로와 반응 종류를 입력한다.

GraphDataset은 1차 전처리를 수행해 구조 반응 시간이력 y를 3차원 벡터로 변환하여 LSTM 모델이 사용하기 쉽게 하고, 해당 구조물의 건물 층수와 반응 종류를 저장한다.

In [1]:
from Utils.dataset import GraphDataset

ChiCHi_folder_dir = "./Data/Linear_Analysis/eval/ChiChi"
NGAWest2_folder_dir = "./Data/Linear_Analysis/eval/NGAWest2"

sampleDataset = GraphDataset(folder_path_list = [ChiCHi_folder_dir, NGAWest2_folder_dir], response_type = "Velocity", numOfData_per_folder = 5)

print(sampleDataset[0])
print(f"\n # of data:{len(sampleDataset)}")

Loading data from ./Data/Linear_Analysis/eval/ChiChi


100%|██████████| 5/5 [00:04<00:00,  1.06it/s]


Loading data from ./Data/Linear_Analysis/eval/NGAWest2


100%|██████████| 5/5 [00:02<00:00,  2.16it/s]

  number of effective data: 10 
Data(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=6000, sample_rate=0.005, ground_motion_name='1999.09.21_01.47.159_Chichi_TCU011_A900_90.0', response_type='Velocity', story=6, y=[1, 2000, 8])

 # of data:10


## Normalization dictionary
데이터를 모델에 입력하기 전에 정규화를 거쳐 각 feature 차원을 [-1, 1]로 조정해야 하므로, 각 feature 차원의 최대값이 필요하다.

In [ ]:
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")
normalized_item_dict = sampleDataset.get_normalized_item_dict()
print("\nnormalization dictionary: \n", normalized_item_dict)


(source) normalization state: False

(target) normalization state: False

normalization dictionary: 
 {'x': {'XYZ_gridline_num': tensor(8.), 'XYZ_grid_index': tensor(7.), 'period': tensor(1.2711), 'DOF': tensor(1.), 'mass': tensor(0.0252), 'XYZ_inertia': tensor(255288.), 'XYZ_mode_shape': tensor(2.1000)}, 'ground_motion': tensor(6140.5513), 'y': tensor(2678.), 'edge_attr': {'S_y': tensor(3687090.), 'S_z': tensor(3687090.), 'area': tensor(30774.), 'element_length': tensor(8000.)}, 'response_type': 'Velocity'}


### Noramalize
각 feature 차원의 최대값을 사용하여 데이터를 정규화한다.

In [10]:
sampleDataset.normalize_source(normalized_item_dict)
sampleDataset.normalize_target(normalized_item_dict)
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")

(source) normalization state: True

(target) normalization state: True


### Denormalize
모델 계산이 끝난 후 계산 결과를 원래 스케일로 역정규화한다.

In [11]:
sampleDataset.denormalize_source(normalized_item_dict)
sampleDataset.denormalize_target(normalized_item_dict)
print(f"(source) normalization state: {sampleDataset.source_norm_state}")
print(f"\n(target) normalization state: {sampleDataset.target_norm_state}")

(source) normalization state: False

(target) normalization state: False


# DataLoader (PyG)
DataLoader를 반복자로 사용하며, 학습 시에는 batch size를 자유롭게 조정하고 shuffle=True로 설정한다. 예측 시에는 batch size=1로 하고 shuffle=False로 설정한다.

In [12]:
from torch_geometric.loader import DataLoader as GraphDataLoader
exclude_keys=["sample_rate", "ground_motion_name", "response_type"]
sample_loader = GraphDataLoader(sampleDataset, batch_size=1, shuffle=False, )

for (i, data) in enumerate(sample_loader):
    print('=======')
    print(f'mini batch {i}:' )
    # data = data.to("cuda")
    print(data)
    print()

mini batch 0:
DataBatch(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[147], ptr=[2])

mini batch 1:
DataBatch(x=[210, 15], edge_index=[2, 904], edge_attr=[904, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[210], ptr=[2])

mini batch 2:
DataBatch(x=[168, 15], edge_index=[2, 742], edge_attr=[742, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[168], ptr=[2])

mini batch 3:
DataBatch(x=[105, 15], edge_index=[2, 444], edge_attr=[444, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[105], ptr=[2])

mini batch 4:
DataBatch(x=[245, 15], edge_index=[2, 1116], edge_attr=[1116, 6], 

## Pack and Pad

In [13]:
compression_rate = 20
data = next(iter(sample_loader))
ground_motion = data.ground_motion.reshape(-1, int(data.ground_motion.size(1)/compression_rate), compression_rate)
ground_motion = ground_motion.to("cuda")

packed_length_list = data.time_steps / compression_rate
total_length = ground_motion.size(1)
print('=======')
print("mini-batch:")
print(data)
print()

print('=======')
print("size of compressed ground motion:")
print(ground_motion.size())
print()
print("PPS length list:")
print(packed_length_list)
print()

packed_x = nn.utils.rnn.pack_padded_sequence(ground_motion, lengths = packed_length_list, batch_first=True, enforce_sorted=False)
seq_unpacked_x, lens_unpacked = nn.utils.rnn.pad_packed_sequence(packed_x, batch_first=True, total_length = total_length)

print('=======')
print("size of unpacked ground motion:")
print(seq_unpacked_x.size())
print()
print("check x has unpacked to original format")
print(torch.equal(seq_unpacked_x, ground_motion))
print()
print("unpack length list:")
print(lens_unpacked)


mini-batch:
DataBatch(x=[147, 15], edge_index=[2, 636], edge_attr=[636, 6], ground_motion=[1, 20000], time_steps=[1], sample_rate=[1], ground_motion_name=[1], response_type=[1], story=[1], y=[1, 2000, 8], batch=[147], ptr=[2])

size of compressed ground motion:
torch.Size([1, 1000, 20])

PPS length list:
tensor([300.])

size of unpacked ground motion:
torch.Size([1, 1000, 20])

check x has unpacked to original format
True

unpack length list:
tensor([300])
